# NS SHOP+ 판매실적 예측을 통한 편성 최적화 방안 도출

### -팀명: B526

# 1. Data Preprocessing

#  1) Load Data

- 분석에 필요한 패키지 설치

In [374]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import KFold

- 모델 학습을 위한 Train Data 불러오기
- Train Data에서 필요 없는 변수 제거하기

In [375]:
df=pd.read_excel('train_925.xlsx')
print(df.columns)
x=df.drop(['Unnamed: 0', 'X', 'index', '방송일시', 'Date', 'year','min','exposure_time','M_code', 'M_new', 'Product_code', 
           'P_new', '상품명', 'group', 'sub', 'subsub',  'total_sales', 'cumsum'], axis=1)
y=df['cumsum']
x.info()

Index(['Unnamed: 0', 'X', 'index', '방송일시', 'Date', 'year', 'day', 'min',
       'month', 'time', 'week', 'weekday', 'holiday', 'exposure_time',
       'cum_expoT', 'M_code', 'M_new', 'Product_code', 'P_new', 'group0',
       'group1', 'group2', '상품명', 'group', 'sub', 'subsub', 'unit_price',
       'ilsibul', 'temp', 'cold_wave', 'heat_wave', 'precipitation', 'package',
       'viewingrate', 'total_sales', 'cumsum'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35750 entries, 0 to 35749
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   day            35750 non-null  int64  
 1   month          35750 non-null  int64  
 2   time           35750 non-null  int64  
 3   week           35750 non-null  int64  
 4   weekday        35750 non-null  int64  
 5   holiday        35750 non-null  int64  
 6   cum_expoT      35750 non-null  float64
 7   group0         35750 non-null  int64  
 8   grou

- 학습을 적용시킬 Test Data 불러오기 
- Test Data에서 필요 없는 변수 제거하기

In [376]:
df2=pd.read_excel('real_test2020.xlsx')
print(df2.columns)
X_test2=df2.drop(['index', '방송일시', 'Date', 'year', 'min', '마더코드', 'M_new', '상품코드', 'P_new'
                 ,'상품명', '상품군', 'sub', 'subsub' ],axis=1)
print(X_test2.info())

Index(['index', '방송일시', 'Date', 'year', 'day', 'min', 'month', 'time', 'week',
       'weekday', 'holiday', 'cum_expoT', '마더코드', 'M_new', '상품코드', 'P_new',
       'group0', 'group1', 'group2', '상품명', '상품군', 'sub', 'subsub',
       'unit_price', 'ilsibul', 'temp', 'cold_wave', 'heat_wave',
       'precipitation', 'package'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2716 entries, 0 to 2715
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   day            2716 non-null   int64  
 1   month          2716 non-null   int64  
 2   time           2716 non-null   int64  
 3   week           2716 non-null   int64  
 4   weekday        2716 non-null   int64  
 5   holiday        2716 non-null   int64  
 6   cum_expoT      2716 non-null   float64
 7   group0         2716 non-null   int64  
 8   group1         2716 non-null   int64  
 9   group2         2716 non-null   int64  
 10  unit_price   

- 2019년 Data를 4:1 비율로 train과 validation으로 분할함
- 파이썬 내 train_test_split 함수 사용

In [378]:
X_train, X_validation, y_train, y_validation=train_test_split(x, y, test_size=0.2,random_state=121)
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28600 entries, 35382 to 11586
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   day            28600 non-null  int64  
 1   month          28600 non-null  int64  
 2   time           28600 non-null  int64  
 3   week           28600 non-null  int64  
 4   weekday        28600 non-null  int64  
 5   holiday        28600 non-null  int64  
 6   cum_expoT      28600 non-null  float64
 7   group0         28600 non-null  int64  
 8   group1         28600 non-null  int64  
 9   group2         28600 non-null  int64  
 10  unit_price     28600 non-null  float64
 11  ilsibul        28600 non-null  int64  
 12  temp           28600 non-null  float64
 13  cold_wave      28600 non-null  int64  
 14  heat_wave      28600 non-null  int64  
 15  precipitation  28600 non-null  float64
 16  package        28600 non-null  float64
 17  viewingrate    28600 non-null  float64
dtypes:

- 범주형 변수를 정제하기 위해 'make_column_transformer' 및 'OneHotEncoder' 함수 사용

In [379]:
transformer = make_column_transformer(

    (OneHotEncoder(), ['day','month','time','week','weekday','holiday','group0','group1','group2','ilsibul','cold_wave','heat_wave']),
    remainder='passthrough')

transformer.fit(x)
x=transformer.transform(x)
X_train= transformer.transform(X_train)
X_validation= transformer.transform(X_validation)

In [380]:
print(X_train.shape)
print(X_validation.shape)

(28600, 170)
(7150, 170)


# 2) Predict viewingrate in 2020 

###### 2020 데이터에 시청률이 없으므로 2019 데이터로 훈련시켜 시청률 예측하여 넣어주어야 함.

In [381]:
df=pd.read_excel('train_925.xlsx')
print(df.columns)
x_view=df.drop(['Unnamed: 0', 'X', 'index', '방송일시', 'Date', 'year','min','exposure_time','M_code', 'M_new', 'Product_code', 
           'P_new', '상품명', 'group', 'sub', 'subsub',  'total_sales', 'cumsum','viewingrate'], axis=1)
y_view=df.viewingrate
x_view.info()

Index(['Unnamed: 0', 'X', 'index', '방송일시', 'Date', 'year', 'day', 'min',
       'month', 'time', 'week', 'weekday', 'holiday', 'exposure_time',
       'cum_expoT', 'M_code', 'M_new', 'Product_code', 'P_new', 'group0',
       'group1', 'group2', '상품명', 'group', 'sub', 'subsub', 'unit_price',
       'ilsibul', 'temp', 'cold_wave', 'heat_wave', 'precipitation', 'package',
       'viewingrate', 'total_sales', 'cumsum'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35750 entries, 0 to 35749
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   day            35750 non-null  int64  
 1   month          35750 non-null  int64  
 2   time           35750 non-null  int64  
 3   week           35750 non-null  int64  
 4   weekday        35750 non-null  int64  
 5   holiday        35750 non-null  int64  
 6   cum_expoT      35750 non-null  float64
 7   group0         35750 non-null  int64  
 8   grou

In [382]:
X_train2, X_validation2, y_train2, y_validation2=train_test_split(x_view, y_view, test_size=0.2,random_state=121)
print(X_train2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28600 entries, 35382 to 11586
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   day            28600 non-null  int64  
 1   month          28600 non-null  int64  
 2   time           28600 non-null  int64  
 3   week           28600 non-null  int64  
 4   weekday        28600 non-null  int64  
 5   holiday        28600 non-null  int64  
 6   cum_expoT      28600 non-null  float64
 7   group0         28600 non-null  int64  
 8   group1         28600 non-null  int64  
 9   group2         28600 non-null  int64  
 10  unit_price     28600 non-null  float64
 11  ilsibul        28600 non-null  int64  
 12  temp           28600 non-null  float64
 13  cold_wave      28600 non-null  int64  
 14  heat_wave      28600 non-null  int64  
 15  precipitation  28600 non-null  float64
 16  package        28600 non-null  float64
dtypes: float64(5), int64(12)
memory usage: 3.9 MB


- 범주형 변수를 정제하기 위해 'make_column_transformer' 및 'OneHotEncoder' 함수 사용

In [383]:
transformer2= make_column_transformer(

    (OneHotEncoder(), ['day','month','time','week','weekday','holiday','group0','group1','group2','ilsibul','cold_wave','heat_wave']),
    remainder='passthrough')

transformer2.fit(x_view)
x_view=transformer2.transform(x_view)
X_train2= transformer2.transform(X_train2)
X_validation2= transformer2.transform(X_validation2)
X_test2= transformer2.transform(X_test2)

- 원핫인코딩 후 shape 확인

In [384]:
print(X_train2.shape)
print(X_validation2.shape)
print(X_test2.shape)

(28600, 169)
(7150, 169)
(2716, 169)


- 시청률 예측에 가장 적절한 모델 결정을 위해 디폴트 상태에서 Gradient Boosting Model, XGBoost, LightGBM을 돌린 후 Best Score 확인
- 가장 Best Sore가 높은 XGBoost 

In [277]:
## 하이퍼파라미터

gb_reg=GradientBoostingRegressor()
xgb_reg=XGBRegressor()
lgb_reg=LGBMRegressor() 

In [278]:
gb_reg.fit(X_train2, y_train2)
print("best score:{0:.4f}".format(gb_reg.score(X_validation2,y_validation2)))

best score:0.1222


In [279]:
xgb_reg.fit(X_train2, y_train2)
print("best score:{0:.4f}".format(xgb_reg.score(X_validation2,y_validation2)))

best score:0.4079


In [280]:
lgb_reg.fit(X_train2, y_train2)
print("best score:{0:.4f}".format(lgb_reg.score(X_validation2,y_validation2)))

best score:0.2532


In [60]:
#xgboost
params={
    'n_estimators':list(range(100,1500,100)),
    'max_depth':list(range(10,32,1)),
}

xgb_reg=XGBRegressor(random_state=121,n_jobs=-1)
grid_cv=GridSearchCV(xgb_reg,param_grid=params,cv=2,n_jobs=-1)
grid_cv.fit(X_train,y_train)
print("최적파라미터:\n",grid_cv.best_params_)
print("최고예측정확도:{0:.4f}".format(grid_cv.best_score_))

최적파라미터:
 {'max_depth': 10, 'n_estimators': 100}
최고예측정확도:0.7497


In [385]:
xgb_reg=XGBRegressor(max_depth= 10,n_estimators= 100)
xgb_reg.fit(X_train2, y_train2)
xgb_reg.score(X_validation2,y_validation2)
y_pred_view=xgb_reg.predict(X_test2) #여기서 y_pred가 2020년 viewingrate 예측치
y_pred_view=np.maximum(0,y_pred_view)

- 위에서 추정한 2020 시청률을 2020년 데이터에 붙임

In [386]:
df2=pd.read_excel('real_test2020.xlsx')
print(df2.columns)
X_test=df2.drop(['index', '방송일시', 'Date', 'year', 'min', '마더코드', 'M_new', '상품코드', 'P_new'
                 ,'상품명', '상품군', 'sub', 'subsub' ],axis=1)

Index(['index', '방송일시', 'Date', 'year', 'day', 'min', 'month', 'time', 'week',
       'weekday', 'holiday', 'cum_expoT', '마더코드', 'M_new', '상품코드', 'P_new',
       'group0', 'group1', 'group2', '상품명', '상품군', 'sub', 'subsub',
       'unit_price', 'ilsibul', 'temp', 'cold_wave', 'heat_wave',
       'precipitation', 'package'],
      dtype='object')


In [387]:
y_pred_view=pd.DataFrame(y_pred_view)

In [388]:
X_test=pd.concat([X_test, y_pred_view], axis=1)
X_test.rename(columns={0:"viewingrate"}, inplace=True)

In [390]:
X_test= transformer.transform(X_test)
X_test.shape

(2716, 170)

# 2. Modeling

# 1) Hyperparameter Setting 

- 파이썬 내 'GridSearchCV' 패키지를 이용하여 최고예측정확도를 가지는 하이퍼 파라미터를 구함

1) Gradient Boosting Model

In [117]:
params={
    'n_estimators':list(range(500,1500,100)),
    'max_depth':list(range(10,32,1)),
}

gb_reg=XGBRegressor(random_state=121,n_jobs=-1)
grid_cv1=GridSearchCV(gb_reg,param_grid=params,cv=2,n_jobs=-1)
grid_cv1.fit(X_train,y_train)
print("최적파라미터:\n",grid_cv1.best_params_)
print("최고예측정확도:{0:.4f}".format(grid_cv1.best_score_))

최적파라미터:
 {'max_depth': 10, 'n_estimators': 600}
최고예측정확도:0.7481


2) XGBoost

In [118]:
params={
    'n_estimators':list(range(500,1500,100)),
    'max_depth':list(range(5,20,1)),
    'min_child_weight':list(range(1,10,1))
}

xgb_reg=XGBRegressor(random_state=121,n_jobs=-1)
grid_cv2=GridSearchCV(xgb_reg,param_grid=params,cv=2,n_jobs=-1)
grid_cv2.fit(X_train,y_train)
print("최적파라미터:\n",grid_cv2.best_params_)
print("최고예측정확도:{0:.4f}".format(grid_cv2.best_score_))

최적파라미터:
 {'max_depth': 7, 'min_child_weight': 2, 'n_estimators': 500}
최고예측정확도:0.7597


3) LightGBM

In [116]:
params={
    'n_estimators':list(range(1000,1500,100)),
    'max_depth':list(range(10,32,1)),
    'num_leaves':list(range(10,30,2))
    
}
 
lgb_reg=LGBMRegressor(random_state=121,n_jobs=-1)
grid_cv3=GridSearchCV(lgb_reg,param_grid=params,cv=2,n_jobs=-1)
grid_cv3.fit(X_train,y_train)
print("최적파라미터:\n",grid_cv3.best_params_)
print("최고예측정확도:{0:.4f}".format(grid_cv3.best_score_))

최적파라미터:
 {'max_depth': 12, 'n_estimators': 1300, 'num_leaves': 26}
최고예측정확도:0.7751


- 최고 예측 정확도가 가장 높은 하이퍼 파라미터를 선택하여 모델에 적용 후 학습시키고자 함
- 각 모형에 대한 Regressor 함수를 미리 이름으로 지정하여 편리하게 학습시키고자 함

# 2) 모델 학습

- Gradient Boosting Model, XGBoost, LightGBM 단일 모형과 두개씩 묶은 앙상블, 세개를 모두 묶은 앙상블 모형 학습

In [123]:
gb_reg=GradientBoostingRegressor(random_state=121,max_depth=10,n_estimators=600) 
xgb_reg=XGBRegressor(random_state=121,max_depth=7,min_child_weight=2,n_estimators=500) 
lgb_reg=LGBMRegressor(random_state=121,max_depth=12,n_estimators=1300,num_leaves=26) 

1) Gradient Boosting Model

In [124]:
gb_reg.fit(X_train, y_train)
y_pred=gb_reg.predict(X_validation)

a1=abs((y_pred-y_validation)/(y_validation))
print("MAPE:{0:.4f}".format(sum(a1)/7150))
print("Best Score:{0:.4f}".format(gb_reg.score(X_validation, y_validation)))

MAPE:0.5922
Best Score:0.8173


2) XGBoost 

In [127]:
xgb_reg.fit(X_train, y_train, eval_set=[(X_validation, y_validation)], early_stopping_rounds=100)
y_pred=xgb_reg.predict(X_validation)

a1=abs((y_pred-y_validation)/(y_validation))
print("MAPE:{0:.4f}".format(sum(a1)/7150))
print("Best Score:{0:.4f}".format(xgb_reg.score(X_validation, y_validation)))

[0]	validation_0-rmse:0.29456
Will train until validation_0-rmse hasn't improved in 100 rounds.
[1]	validation_0-rmse:0.21105
[2]	validation_0-rmse:0.15439
[3]	validation_0-rmse:0.11651
[4]	validation_0-rmse:0.09198
[5]	validation_0-rmse:0.07681
[6]	validation_0-rmse:0.06800
[7]	validation_0-rmse:0.06299
[8]	validation_0-rmse:0.06009
[9]	validation_0-rmse:0.05858
[10]	validation_0-rmse:0.05767
[11]	validation_0-rmse:0.05702
[12]	validation_0-rmse:0.05645
[13]	validation_0-rmse:0.05589
[14]	validation_0-rmse:0.05561
[15]	validation_0-rmse:0.05540
[16]	validation_0-rmse:0.05519
[17]	validation_0-rmse:0.05490
[18]	validation_0-rmse:0.05461
[19]	validation_0-rmse:0.05435
[20]	validation_0-rmse:0.05345
[21]	validation_0-rmse:0.05333
[22]	validation_0-rmse:0.05319
[23]	validation_0-rmse:0.05303
[24]	validation_0-rmse:0.05292
[25]	validation_0-rmse:0.05282
[26]	validation_0-rmse:0.05276
[27]	validation_0-rmse:0.05266
[28]	validation_0-rmse:0.05254
[29]	validation_0-rmse:0.05238
[30]	validatio

[258]	validation_0-rmse:0.04367
[259]	validation_0-rmse:0.04364
[260]	validation_0-rmse:0.04366
[261]	validation_0-rmse:0.04365
[262]	validation_0-rmse:0.04365
[263]	validation_0-rmse:0.04364
[264]	validation_0-rmse:0.04363
[265]	validation_0-rmse:0.04365
[266]	validation_0-rmse:0.04362
[267]	validation_0-rmse:0.04362
[268]	validation_0-rmse:0.04359
[269]	validation_0-rmse:0.04361
[270]	validation_0-rmse:0.04362
[271]	validation_0-rmse:0.04364
[272]	validation_0-rmse:0.04364
[273]	validation_0-rmse:0.04365
[274]	validation_0-rmse:0.04364
[275]	validation_0-rmse:0.04364
[276]	validation_0-rmse:0.04362
[277]	validation_0-rmse:0.04361
[278]	validation_0-rmse:0.04360
[279]	validation_0-rmse:0.04360
[280]	validation_0-rmse:0.04360
[281]	validation_0-rmse:0.04360
[282]	validation_0-rmse:0.04359
[283]	validation_0-rmse:0.04360
[284]	validation_0-rmse:0.04359
[285]	validation_0-rmse:0.04359
[286]	validation_0-rmse:0.04358
[287]	validation_0-rmse:0.04360
[288]	validation_0-rmse:0.04358
[289]	va

3) LightGBM

In [129]:
lgb_reg.fit(X_train, y_train, eval_set=[(X_validation, y_validation)])
y_pred=lgb_reg.predict(X_validation)

a1=abs((y_pred-y_validation)/(y_validation))
print("MAPE:{0:.4f}".format(sum(a1)/7150))
print("Best Score:{0:.4f}".format(lgb_reg.score(X_validation, y_validation)))

[1]	valid_0's l2: 0.00930952
[2]	valid_0's l2: 0.00844526
[3]	valid_0's l2: 0.00773636
[4]	valid_0's l2: 0.00714899
[5]	valid_0's l2: 0.00666507
[6]	valid_0's l2: 0.00625893
[7]	valid_0's l2: 0.00592072
[8]	valid_0's l2: 0.00564129
[9]	valid_0's l2: 0.00540139
[10]	valid_0's l2: 0.00516543
[11]	valid_0's l2: 0.00497784
[12]	valid_0's l2: 0.0048317
[13]	valid_0's l2: 0.00467813
[14]	valid_0's l2: 0.00455169
[15]	valid_0's l2: 0.00443599
[16]	valid_0's l2: 0.00434367
[17]	valid_0's l2: 0.00425671
[18]	valid_0's l2: 0.00418915
[19]	valid_0's l2: 0.00409507
[20]	valid_0's l2: 0.004029
[21]	valid_0's l2: 0.00397333
[22]	valid_0's l2: 0.00392041
[23]	valid_0's l2: 0.00384905
[24]	valid_0's l2: 0.00380976
[25]	valid_0's l2: 0.00375495
[26]	valid_0's l2: 0.00369728
[27]	valid_0's l2: 0.00366301
[28]	valid_0's l2: 0.00360511
[29]	valid_0's l2: 0.00356989
[30]	valid_0's l2: 0.00353779
[31]	valid_0's l2: 0.00350026
[32]	valid_0's l2: 0.00347387
[33]	valid_0's l2: 0.00343886
[34]	valid_0's l2: 0.0

[272]	valid_0's l2: 0.00228093
[273]	valid_0's l2: 0.00228053
[274]	valid_0's l2: 0.00228034
[275]	valid_0's l2: 0.0022793
[276]	valid_0's l2: 0.00227776
[277]	valid_0's l2: 0.00227663
[278]	valid_0's l2: 0.00227532
[279]	valid_0's l2: 0.00227462
[280]	valid_0's l2: 0.00227277
[281]	valid_0's l2: 0.00227141
[282]	valid_0's l2: 0.00227043
[283]	valid_0's l2: 0.00227042
[284]	valid_0's l2: 0.00227001
[285]	valid_0's l2: 0.0022699
[286]	valid_0's l2: 0.00226857
[287]	valid_0's l2: 0.00226763
[288]	valid_0's l2: 0.00226691
[289]	valid_0's l2: 0.00226514
[290]	valid_0's l2: 0.00226363
[291]	valid_0's l2: 0.00226036
[292]	valid_0's l2: 0.00225933
[293]	valid_0's l2: 0.00225865
[294]	valid_0's l2: 0.00225807
[295]	valid_0's l2: 0.00225676
[296]	valid_0's l2: 0.00225578
[297]	valid_0's l2: 0.00225477
[298]	valid_0's l2: 0.00225432
[299]	valid_0's l2: 0.00225306
[300]	valid_0's l2: 0.00225244
[301]	valid_0's l2: 0.00225224
[302]	valid_0's l2: 0.00225148
[303]	valid_0's l2: 0.00225085
[304]	vali

[555]	valid_0's l2: 0.00202131
[556]	valid_0's l2: 0.00202009
[557]	valid_0's l2: 0.00201959
[558]	valid_0's l2: 0.00201954
[559]	valid_0's l2: 0.00201836
[560]	valid_0's l2: 0.00201805
[561]	valid_0's l2: 0.00201579
[562]	valid_0's l2: 0.00201495
[563]	valid_0's l2: 0.00201504
[564]	valid_0's l2: 0.00201482
[565]	valid_0's l2: 0.00201411
[566]	valid_0's l2: 0.00201309
[567]	valid_0's l2: 0.00201247
[568]	valid_0's l2: 0.00201208
[569]	valid_0's l2: 0.00201173
[570]	valid_0's l2: 0.00201124
[571]	valid_0's l2: 0.00201106
[572]	valid_0's l2: 0.00200927
[573]	valid_0's l2: 0.00200916
[574]	valid_0's l2: 0.00200878
[575]	valid_0's l2: 0.00200816
[576]	valid_0's l2: 0.00200821
[577]	valid_0's l2: 0.00200782
[578]	valid_0's l2: 0.00200703
[579]	valid_0's l2: 0.00200663
[580]	valid_0's l2: 0.00200609
[581]	valid_0's l2: 0.00200553
[582]	valid_0's l2: 0.00200511
[583]	valid_0's l2: 0.00200411
[584]	valid_0's l2: 0.00200346
[585]	valid_0's l2: 0.00200343
[586]	valid_0's l2: 0.00200291
[587]	va

[835]	valid_0's l2: 0.00189076
[836]	valid_0's l2: 0.00189081
[837]	valid_0's l2: 0.00189002
[838]	valid_0's l2: 0.00188992
[839]	valid_0's l2: 0.00188988
[840]	valid_0's l2: 0.00188983
[841]	valid_0's l2: 0.00188927
[842]	valid_0's l2: 0.00188893
[843]	valid_0's l2: 0.00188847
[844]	valid_0's l2: 0.00188831
[845]	valid_0's l2: 0.00188767
[846]	valid_0's l2: 0.00188701
[847]	valid_0's l2: 0.00188645
[848]	valid_0's l2: 0.00188654
[849]	valid_0's l2: 0.00188575
[850]	valid_0's l2: 0.00188531
[851]	valid_0's l2: 0.00188489
[852]	valid_0's l2: 0.00188474
[853]	valid_0's l2: 0.00188417
[854]	valid_0's l2: 0.00188375
[855]	valid_0's l2: 0.00188328
[856]	valid_0's l2: 0.00188302
[857]	valid_0's l2: 0.0018826
[858]	valid_0's l2: 0.00188277
[859]	valid_0's l2: 0.00188238
[860]	valid_0's l2: 0.00188175
[861]	valid_0's l2: 0.00188146
[862]	valid_0's l2: 0.00188104
[863]	valid_0's l2: 0.00188097
[864]	valid_0's l2: 0.00188041
[865]	valid_0's l2: 0.00188021
[866]	valid_0's l2: 0.0018798
[867]	vali

[1123]	valid_0's l2: 0.00181693
[1124]	valid_0's l2: 0.00181701
[1125]	valid_0's l2: 0.00181689
[1126]	valid_0's l2: 0.00181678
[1127]	valid_0's l2: 0.00181643
[1128]	valid_0's l2: 0.00181621
[1129]	valid_0's l2: 0.00181618
[1130]	valid_0's l2: 0.00181542
[1131]	valid_0's l2: 0.00181523
[1132]	valid_0's l2: 0.00181493
[1133]	valid_0's l2: 0.00181475
[1134]	valid_0's l2: 0.00181415
[1135]	valid_0's l2: 0.0018138
[1136]	valid_0's l2: 0.00181383
[1137]	valid_0's l2: 0.00181368
[1138]	valid_0's l2: 0.00181344
[1139]	valid_0's l2: 0.00181325
[1140]	valid_0's l2: 0.00181259
[1141]	valid_0's l2: 0.00181254
[1142]	valid_0's l2: 0.00181249
[1143]	valid_0's l2: 0.00181198
[1144]	valid_0's l2: 0.00181147
[1145]	valid_0's l2: 0.0018112
[1146]	valid_0's l2: 0.00181092
[1147]	valid_0's l2: 0.00180955
[1148]	valid_0's l2: 0.00180945
[1149]	valid_0's l2: 0.00180919
[1150]	valid_0's l2: 0.00180877
[1151]	valid_0's l2: 0.00180836
[1152]	valid_0's l2: 0.00180826
[1153]	valid_0's l2: 0.00180821
[1154]	val

4) GB & XGB

In [132]:
named_estimators = [
    ("gb_reg", gb_reg),
    ("xgb_reg", xgb_reg)
]

vr2_1 = VotingRegressor(named_estimators)
vr2_1.fit(X_train, y_train)
y_pred2_1=vr2_1.predict(X_validation)
a2_1=abs((y_pred2_1-y_validation)/y_validation)
 
print("MAPE:{0:.4f}".format(sum(a2_1)/7150))
print("Best Score:{0:.4f}".format(vr2_1.score(X_validation, y_validation)))

MAPE:0.5873
Best Score:0.8292


5) GB & LGB

In [133]:
named_estimators = [
    ("gb_reg", gb_reg),
    ("lgb_reg", lgb_reg)
]

vr2_2 = VotingRegressor(named_estimators)
vr2_2.fit(X_train, y_train)
y_pred2_2=vr2_2.predict(X_validation)
a2_2=abs((y_pred2_2-y_validation)/y_validation)
 
print("MAPE:{0:.4f}".format(sum(a2_2)/7150))
print("Best Score:{0:.4f}".format(vr2_2.score(X_validation, y_validation)))

MAPE:0.6030
Best Score:0.8349


6) XGB & LGB

In [134]:
named_estimators = [
    ("xgb_reg", xgb_reg),
    ("lgb_reg", lgb_reg)
]

vr2_3 = VotingRegressor(named_estimators)
vr2_3.fit(X_train, y_train)
y_pred2_3=vr2_3.predict(X_validation)
a2_3=abs((y_pred2_3-y_validation)/y_validation)
 
print("MAPE:{0:.4f}".format(sum(a2_3)/7150))
print("Best Score:{0:.4f}".format(vr2_3.score(X_validation, y_validation)))

MAPE:0.6332
Best Score:0.8350


7) GB & XGB & LGB

In [135]:
named_estimators = [
    ("gb_reg", gb_reg),
    ("xgb_reg", xgb_reg),
    ("lgb_reg", lgb_reg)
]

vr3 = VotingRegressor(named_estimators)
vr3.fit(X_train, y_train)
y_pred3=vr3.predict(X_validation)
a3=abs((y_pred3-y_validation)/y_validation)
 
print("MAPE:{0:.4f}".format(sum(a3)/7150))
print("Best Score:{0:.4f}".format(vr3.score(X_validation, y_validation)))

MAPE:0.5949
Best Score:0.8368


# 3) Stacking

- 예측 모형의 성능을 향상시키기 위해 학습시킨 앙상블 모델들 중 가장 우수한 모델 3개를 골라 stacking 사용
- Stacking을 사용해서 2020년 6월의 상품별 누적합의 예측치를 구함

In [422]:
from sklearn.model_selection import KFold
def get_stacking_data(model, X_train,y_train, X_val,X_test, n_folds=10):
    kfold=KFold(n_splits=n_folds, random_state=None)
    
    train_fold_predict=np.zeros((X_train.shape[0],1))
    test_predict1=np.zeros((X_val.shape[0], n_folds))
    test_predict2=np.zeros((X_test.shape[0], n_folds))
    print("model :", model.__class__.__name__)
    
    for cnt, (train_index , valid_index) in enumerate(kfold.split(X_train,y_train)):
        X_train_=X_train[train_index]
        y_train_=y_train[train_index]
        X_validation=X_train[valid_index]
        y_validation=y_train[valid_index]
        
        model.fit(X_train_, y_train_)
        
        train_fold_predict[valid_index,:]=model.predict(X_validation).reshape(-1,1)
        
        test_predict1[:,cnt]=model.predict(X_val)
        test_predict2[:,cnt]=model.predict(X_test)
        
    test_predict_mean1= np.mean(test_predict1,axis=1).reshape(-1,1)
    test_predict_mean2= np.mean(test_predict2,axis=1).reshape(-1,1)
    return train_fold_predict, test_predict_mean1, test_predict_mean2
    

In [423]:
XX=X_train.toarray()
XX.shape
yy=pd.Series.to_numpy(y_train)
yy.shape

(28600,)

In [424]:
vr2_1train, vr2_1val, vr2_1test=get_stacking_data(vr2_1, XX,yy, X_validation, X_test, n_folds=10)
vr2_2train, vr2_2val, vr2_2test=get_stacking_data(vr2_2, XX,yy, X_validation, X_test, n_folds=10)
vr2_3train, vr2_3val, vr2_3test=get_stacking_data(vr2_3, XX,yy, X_validation, X_test, n_folds=10)

new_X_train=np.concatenate((vr2_1train,vr2_2train,vr2_3train), axis=1)
new_X_val=np.concatenate((vr2_1val,vr2_2val,vr2_3val), axis=1)
new_X_test=np.concatenate((vr2_1test,vr2_2test,vr2_3test), axis=1)

model : VotingRegressor
model : VotingRegressor
model : VotingRegressor


- 스태킹으로 구한 예측치와 y_validation 사이의 MAPE를 구함

In [426]:
gb_reg.fit(new_X_train, y_train)
score=gb_reg.score(new_X_val, y_validation)
stack_pred=gb_reg.predict(new_X_test)
y_pred=gb_reg.predict(new_X_val)
a=abs((y_pred-y_validation)/y_validation)
mape=sum(a)/7150

print("MAPE:{0:.4f}".format(sum(a)/7150))
print("Best Score:{0:.4f}".format(gb_reg.score(new_X_val, y_validation)))

MAPE:0.6016
Best Score:0.8427


### - 2020년 매출 예측치

In [427]:
a=pd.DataFrame(stack_pred)
a.to_csv("predict_2020.csv")

# 3. 요일별 / 시간별 최적화 방안 찾기

In [430]:
X_test3=pd.read_excel('broadcast_final.xlsx')
print(X_test3.columns)

Index(['index', 'day', 'month', 'time', 'week', 'weekday', 'holiday',
       'cum_expoT', 'Product_group', 'sub', 'subsub', 'group0', 'group1',
       'group2', 'unit_price', 'ilsibul', 'temp', 'cold_wave', 'heat_wave',
       'precipitation', 'package', 'viewingrate'],
      dtype='object')


In [432]:
x_broad=X_test3.drop(['index', 'Product_group', 'sub', 'subsub'], axis=1)

In [434]:
X_broad=transformer.transform(x_broad)

- 요일별 / 시간별 최적화를 위한 cumsum prediction
- 위에서 적합시켰던 스태킹 모델을 사용하고, test data만 방금 불러온 'broadcast' 파일을 사용함.

In [435]:
vr2_1train, vr2_1val, vr2_1test=get_stacking_data(vr2_1, XX,yy, X_validation, X_broad, n_folds=10)
vr2_2train, vr2_2val, vr2_2test=get_stacking_data(vr2_2, XX,yy, X_validation, X_broad, n_folds=10)
vr2_3train, vr2_3val, vr2_3test=get_stacking_data(vr2_3, XX,yy, X_validation, X_broad, n_folds=10)

new_X_test3=np.concatenate((vr2_1test,vr2_2test,vr2_3test), axis=1)

model : VotingRegressor
model : VotingRegressor
model : VotingRegressor


In [436]:
stack_pred3=gb_reg.predict(new_X_test3)

In [437]:
b=pd.DataFrame(stack_pred3)
b.to_csv("broadcast_max.csv")